In [37]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma

In [94]:
def solve_1to1(Φ_i_j):
    n, m = np.shape(Φ_i_j)
    M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
                      [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
    q = np.concatenate((np.ones(n), np.ones(m)))

    model = grb.Model()
    mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
    model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
    model.addConstr(M_z_a @ mu_a <= q)
    
    model.Params.OutputFlag = 0
    model.optimize()

    return np.array(model.x, dtype= bool).reshape([n,m])*1

In [95]:
a = np.ones([3,3])


In [208]:
class OneToOneITU():
    def __init__(self, n, m, size_params, random_seed, lbs=(1, 1), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 

        # generate problem
        np.random.seed(random_seed)
        self.A_ij = np.random.randint(1, size_params[0], size=[n, m])/  np.random.randint(1, size_params[0], size=[n, m])
        # values_A = np.array([1,2,4,5,8,10,16,20])
        # self.A_ij = np.random.choice(values_A[size_params[0]],size= [n,m])
        self.B_ij = np.random.randint(0, size_params[1], size=[n, m])

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            U_ij = self.B_ij[i_idx,:] - self.A_ij[i_idx,:] * V_j[None, :], 
            out = self.lb_U * np.ones(np.shape(U_ij[0])[0])[:,None]
            return np.concatenate( ( U_ij[0], out  ) , axis= 1)
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [700]:
n_small = 5
m_small = 5
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (3,3) , random_seed= 3, lbs =(0,0), tol = 0)

In [212]:
example_small.get_U_ij(V_ex, np.array([1,2]))

array([[1., 0., 3., 1., 1., 0.],
       [3., 0., 2., 3., 1., 0.]])

#### Checking modules

The following functions check equilibrium conditions for a candidate $(\mu, U, V).$. Below I will refer to the equilibrium conditions as follows:
* (*Primal Feasibility*) $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ (and $\mu_{ij} \in \{0,1\}$),
* (*Primal Complementary Slackness*) $D_{ij}(U_i,V_j) = 0$ if $ \mu_{ij}  = 1$, 
* (*Dual Feasibility*) $D_{ij}(U_i,V_j) \geq 0$,
* (*Dual Complementary Slackness*) $U_i =  U_0 $ if $\sum_j \mu_{ij} = 0$ and $V_i =  V_0 $ if $\sum_i \mu_{ij} = 0,$ 
* (*Individual Rationality*) $U_i \geq  U_0 ,$ $V_i \geq  V_0 .$ 

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [213]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.mean((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.mean((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\frac{1}{{n}} \sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\frac{1}{{m}}\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [214]:
def check_dual_feas(self,U_i,V_j, output = None):
 
    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return D_feas, np.where(D_feas<0 )
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.mean()}'))
    
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [215]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    
    if output:
        return P_CS_ij
    else:
        display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.mean()}'))

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [216]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [217]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [218]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

# Algorithm

In [651]:
def iteration(self,i_ent, mu_, V_j_):
    mu = mu_.copy()
    V_j = V_j_.copy()

    unassigned_js = np.concatenate((mu[:,:-1].sum(0) == 0,[True]))

    C  = np.zeros(self.n, dtype= bool)
    C_i_ent = np.zeros(self.n, dtype= bool)
    C[i_ent] = 1
    B = np.ones(self.m, dtype= bool)
    L_j = np.ones(self.m, dtype= int) *(self.n + 1)
    d_j = np.ones(self.m + 1, dtype= int) * np.inf

    iter = 0
    while np.any(C == 1):
        iter += 1
        i = np.argmax(C)
        #print(f"iter {iter}, i: {i}")
        C[i] = 0
        C_i_ent[i] = 1
        U_ij_s = self.get_U_ij(V_j, i)[0]
        π_i = np.max(U_ij_s)
        #print(π_i)

        suboptimal_js = U_ij_s < π_i 
 
        A_ij_star_max = np.max(self.A_ij[i,~(suboptimal_js[:-1] )])
        d_j[suboptimal_js]= np.minimum(d_j[suboptimal_js], (π_i - U_ij_s[suboptimal_js]) /A_ij_star_max) 

       
        unissigned_optimal_js = np.where( unassigned_js  * (π_i == U_ij_s))[0]
    
        if len(unissigned_optimal_js)> 0: #####
            i_t = i
            j_t = unissigned_optimal_js[0]
            #print(j_t)
            # if j_t != self.m:
            #     w_i_s = np.sort( self.get_U_ij(V_j, i_t)[0])[-2]
            #     V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            while i_t != i_ent :
 
                j_t_plus_1 = np.argmax(mu[i_t,: ])
                mu[i_t,: ] = 0
                mu[i_t, j_t] = 1
                ########### print( self.get_U_ij(V_j, i_t)[0] )
                #w_i_s = np.sort( self.get_U_ij(V_j, i_t)[0])[-2]
                #V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)

                j_t = j_t_plus_1
                i_t = L_j[j_t]
         
            w_i_s = np.sort( self.get_U_ij(V_j, i_t)[0])[-2]
            mu[i_t,: ] = 0
            mu[i_t, j_t] = 1
            V_j[j_t] = self.get_V_ij(i_t  , j_t,  w_i_s)
            #print("augmentation")
            return mu, V_j
            
        for j in np.where(B * (~suboptimal_js[:-1]) >0)[0]:
            #if mu[:,j].sum() == 1:
                i_j = np.argmax(mu[:,j])
                B[j] = 0
                C[i_j] = 1
                L_j[j] = i if L_j[j] == self.n + 1 else None
                #print(f"j : {j}")
                #print(L_j)
                #print(f"i:{i} -> j:{j} ")
    
     
    return C_i_ent, B * (d_j[:-1] < np.inf), d_j, iter

OneToOneITU.iteration = iteration

In [652]:
example_small.iteration(0, mu_ex_pre, V_ex_pre )

(array([[1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.]]),
 array([0.5  , 0.125, 4.   , 0.5  , 0.   ]))

In [644]:
example_small.get_U_ij(V_ex_pre, np.ones(example_small.n, dtype= bool))

array([[ 2.75  ,  2.75  , -3.    ,  0.5   ,  0.    ,  0.    ],
       [ 0.75  , -0.25  ,  1.    ,  0.5   ,  1.    ,  0.    ],
       [ 2.    , -0.0625,  0.    ,  2.    ,  1.    ,  0.    ],
       [-0.25  ,  2.75  , -4.    ,  2.75  ,  2.    ,  0.    ],
       [ 0.    ,  0.875 , -8.    ,  2.    ,  2.    ,  0.    ]])

In [626]:
mu_ex_pre

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [1071]:
def interim_auction(self,C, O, out_opt_i, V_j_, mu_ ):
    V_j = V_j_.copy()
    pi_ij = mu_[C,:-1][:,O]
 
    iter = 0
    U_ij = np.concatenate((self.get_U_ij(V_j, C)[:, :-1][:,O], out_opt_i[:,None]), axis= 1)
    sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
    w_i = U_ij[np.arange(C.sum()),sorted_w_id[:,0] ]
    π_i = np.max(U_ij, axis = 1)
    ij_optimal = U_ij == π_i[:, None]

    offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:][:,O] ) * ij_optimal[:,:-1]

    print(U_ij)
    print(offers)
    # not_contested_js = np.concatenate((pi_ij.sum(0) == 0, [True]))
    # print(not_contested_js)
    # sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
    # w_i = np.max(U_ij[:,not_contested_js] , axis= 1)
    # offers_coop = ((π_i - w_i)[:,None]/ self.A_ij[C][:,O] ) * ij_optimal[:,:-1]
  
    while offers.sum() >0 or pi_ij.sum() < C.sum() - 1:
        #print(f"ITER {iter}")
        iter += 1
        if offers.sum() >0:
            V_j[O] += np.max( offers, axis=0 )
         
        else:
     
            not_contested_js = np.concatenate((pi_ij.sum(0) == 0, [True]))
            sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
            w_i = np.max(U_ij[:,not_contested_js] , axis= 1)
  
            offers_coop = ((π_i - w_i)[:,None]/ self.A_ij[C][:,O] ) * ij_optimal[:,:-1]
         
            V_j[O] += np.min( offers_coop[offers_coop > 0] )


        U_ij = np.concatenate((self.get_U_ij(V_j, C)[:, :-1][:, O], out_opt_i[:,None]), axis= 1)
        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        w_i = U_ij[np.arange(C.sum()),sorted_w_id[:,0]]
        π_i = np.max(U_ij, axis = 1)
        ij_optimal = U_ij == π_i[:, None]
      
        offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:][:,O] ) * ij_optimal[:,:-1]
        pi_ij = solve_1to1(ij_optimal[:,:-1])
    

        # if iter >2000:
        #     break


    return pi_ij, V_j, iter

OneToOneITU.interim_auction =interim_auction

In [1507]:
def interim_auction(self,C, O, out_opt_i, V_j_, mu_ ):
    V_j = V_j_.copy()
    pi_ij = np.concatenate((mu_[C,:-1][:,O], np.zeros(C.sum(), dtype= bool)[:,None] ), axis= 1)
    iter = 0
    

    # not_contested_js = np.concatenate((pi_ij.sum(0) == 0, [True]))
    # print(not_contested_js)
    # sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
    # w_i = np.max(U_ij[:,not_contested_js] , axis= 1)
    # offers_coop = ((π_i - w_i)[:,None]/ self.A_ij[C][:,O] ) * ij_optimal[:,:-1]
  
    while True:
        print(f"ITER {iter}")
        iter += 1
        #print(pi_ij)
        

        U_ij = np.concatenate((self.get_U_ij(V_j, C)[:, :-1][:,O], out_opt_i[:,None]), axis= 1)
        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        w_i = U_ij[np.arange(C.sum()),sorted_w_id[:,0] ]
        π_i = np.max(U_ij, axis = 1)
        ij_optimal = U_ij == π_i[:, None]
        offers = ((π_i - w_i)[:,None]/ self.A_ij[C,:][:,O] ) * ij_optimal[:,:-1]

        pi_ij = solve_1to1(ij_optimal)
        if iter > 3:
            print(offers)
            print(U_ij)
            print(pi_ij)
            break
  

        if int(pi_ij.sum()) == int(C.sum())  :
    
            return pi_ij, V_j, iter

        if offers[pi_ij.sum(1) == 0,: ].sum() > 0:
            V_j[O] += np.max( offers[pi_ij.sum(1) == 0,: ], axis=0 )
         
        else:
            not_contested_js = np.concatenate((pi_ij[:,:-1].sum(0) == 0, [True]))
            print(not_contested_js)
            sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
            w_i = np.max(U_ij[:,not_contested_js] , axis= 1)
  
            offers_coop = ((π_i - w_i)[:,None]/ self.A_ij[C][:,O] ) * ij_optimal[:,:-1]
            V_j[O] += np.min( offers_coop[offers_coop > 0] )

        print(V_j[O])
        

OneToOneITU.interim_auction =interim_auction

In [1508]:
example_small.interim_auction(C_i, ~B,w_i_out_B ,V_ex_pre, mu_ex_pre )

ITER 0
[False False False  True]
[0.5 0.5 0.5]
ITER 1
[2.  0.5 0.5]
ITER 2
[False False False  True]
[2.5 1.  1. ]
ITER 3
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[-0.5   0.   -1.    0.  ]
 [ 0.75 -0.5   0.    1.  ]
 [-2.5   2.5   2.5   1.  ]
 [-4.    1.    1.    1.  ]]
[[0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 0]]


In [1479]:
w_i_out_B

array([0., 1., 1., 1.])

In [1439]:
mu_ex[C_i,:-1][:,~B]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [1506]:
example_small.get_U_ij(np.array([2,0,.5,0,.5]) , np.ones(example_small.n, dtype=bool))[C_i,:-1][:,~B]

array([[ 0.  ,  1.  , -0.5 ],
       [ 1.  , -0.25,  1.  ],
       [-2.  ,  2.75,  2.75],
       [-3.  ,  1.5 ,  1.5 ]])

In [1431]:
n_small = 5
m_small = 5
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (3,4) , random_seed= 115, lbs =(0,0), tol = 0)

In [1432]:
mu_ex = np.zeros([n_small,m_small+1])
V_ex = np.ones(example_small.m) * example_small.lb_V

In [1436]:
#C_i = np.zeros(example_small.n, dtype= bool)
for p in range(1):
    i_ent = np.where(mu_ex.sum(1) == 0)[0][0]
    iter_values = example_small.iteration(i_ent, mu_ex, V_ex )
    #print(iter_values[1])
    V_ex_pre = V_ex.copy()
    mu_ex_pre = mu_ex.copy()
    if len(iter_values) == 2:
        mu_ex, V_ex  = iter_values
        print("augment")
 
    else:

        C_i, B, d ,j_low= iter_values
        U_ij = example_small.get_U_ij(V_ex, C_i)
        val_i = np.max(U_ij, axis = 1)

        w_i_out_B = np.max(U_ij[:,np.concatenate((B,[True]))], axis = 1)

       
        #V_ex[~B] += np.min(d[B_with_out])
        pi_ij, V_ex, iter = example_small.interim_auction(C_i, ~B,w_i_out_B ,V_ex, mu_ex )
        id_O = 0
        for j in np.where( B == 0 )[0]:
            mu_ex[C_i,j] = pi_ij[:,id_O]
            id_O += 1
        

ValueError: zero-size array to reduction operation minimum which has no identity

In [1414]:
mu_ex

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [1313]:
example_small.get_U_ij(V_ex, np.ones(example_small.n, dtype=bool))

array([[ 0. , -1. , -2. , -1. ,  1. ,  0. ],
       [ 3. ,  1. ,  0. , -1. ,  0. ,  0. ],
       [ 2. ,  2. ,  1. , -2. ,  1. ,  0. ],
       [ 1. ,  0.5,  1. , -1. ,  0. ,  0. ],
       [ 1. ,  1. , -1. ,  1. ,  0. ,  0. ]])

In [1275]:
V_ex

array([0., 1., 2., 2., 0.])

In [884]:
mu_ex

array([[0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# Coop auction

In [1279]:
n_small = 5
m_small = 5
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (3,4) , random_seed= 1, lbs =(0,0), tol = 0)

In [1337]:
def coop_ITU_auction(self):
    mu_ij = np.zeros([self.n,self.m+1])
    V_j = np.ones(self.m) * self.lb_V
    while np.any( mu_ij.sum() < np.minimum(self.n, self.m)):
    # for p in range(5):
        
        i_ent = np.where(mu_ij.sum(1) == 0)[0][0]
        iter_values = self.iteration(i_ent, mu_ij, V_j )
        #print(iter_values[1])
        V_j_pre = V_j.copy()
        mu_ij_pre = mu_ij.copy()
        if len(iter_values) == 2:
            mu_ij, V_j  = iter_values
            #print("augment")    
        else:

            C_i, B, d ,j_low= iter_values
            U_ij = self.get_U_ij(V_j, C_i)
            val_i = np.max(U_ij, axis = 1)
            w_i_out_B = np.max(U_ij[:,np.concatenate((B,[True]))], axis = 1)
    
            pi_ij, V_j, iter = self.interim_auction(C_i, ~B,w_i_out_B ,V_j, mu_ij )
            id_O = 0
            for j in np.where( B == 0 )[0]:
                mu_ij[C_i,j] = pi_ij[:,id_O]
                id_O += 1

    id_i , id_j  =  np.where(mu_ij[:,:-1] == 1)
    U_i = np.ones(self.n) * self.lb_U
    U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]  
    return mu_ij, U_i, V_j

OneToOneITU.coop_ITU_auction = coop_ITU_auction

In [1338]:
mu_ex , U_ex,  V_ex  = example_small.coop_ITU_auction()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [1339]:
V_ex

array([0., 1., 2., 2., 0.])

In [1340]:
mu_ex

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [1341]:
mu_ex

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [1342]:
example_small.get_U_ij(V_ex, np.ones(example_small.n, dtype=bool))

array([[ 0. , -1. , -2. , -1. ,  1. ,  0. ],
       [ 3. ,  1. ,  0. , -1. ,  0. ,  0. ],
       [ 2. ,  2. ,  1. , -2. ,  1. ,  0. ],
       [ 1. ,  0.5,  1. , -1. ,  0. ,  0. ],
       [ 1. ,  1. , -1. ,  1. ,  0. ,  0. ]])

In [1429]:
id_i , id_j  =  np.where(mu_ex[:,:-1] == 1)
U_ex = np.ones(example_small.n) * example_small.lb_U
U_ex[id_i] = example_small.get_U_ij(V_ex, id_i)[np.arange(len(id_i)), id_j]  

In [1430]:
example_small.check_all((mu_ex[:,:-1] , U_ex , V_ex ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 5 over 5

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [818]:
example_small.check_dual_feas ( U_ex , V_ex , output= True)

(array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 (array([], dtype=int64), array([], dtype=int64)))

In [1418]:
for iter in range(1000000):
    print(iter)
    n_small = 5
    m_small = 5
    example_simul = OneToOneITU( n = n_small,m = m_small, size_params= (3,4) , random_seed= iter, lbs =(0,0), tol = 0)
    mu_sim , U_sim,  V_sim  = example_simul.coop_ITU_auction()
    # print(example_simul.B_ij[0,:])
    mu_sim = mu_sim[:,:-1]
    feas_dual = example_simul. check_dual_feas(U_sim ,V_sim, output = True)[0].sum() 
    CS = example_simul.check_primal_CS(U_sim, V_sim, mu_sim, output= True).sum()
    IR = ((mu_sim.sum(0) == 0) * ( V_sim - example_simul.lb_V)).sum()   
    IR_i = ((mu_sim.sum(1) == 0) * ( U_sim - example_simul.lb_U)).sum() 
    feas = np.all(mu_sim.sum(0)<=1) * np.all(mu_sim.sum(1)<=1)
   
    if CS != 0 or IR != 0 or ~feas or feas_dual!= 0 or IR_i != 0:
        print("porcaccio")
        print(IR)
        print(CS)
        print(feas_dual)
        print(f"iter{iter}")
        break

0
1
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
2
3
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
4
5
6
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
7
8
9
10
11
12
13
14
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
37
38
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zer

ValueError: zero-size array to reduction operation minimum which has no identity